# 📣 Pronóstico de tasa de cancelación de clientes - Telecom

## 📌 Introducción
Al operador de telecomunicaciones Interconnect le gustaría poder pronosticar su tasa de cancelación de clientes. 
    
Si se descubre que un usuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales. 
 

## 🎯 Problema de negocio
La clientela puede elegir entre un pago mensual o firmar un contrato de 1 o 2 años. Puede utilizar varios métodos de pago y recibir una factura electrónica después de una transacción. Telecom necesita descubrir si un usuario o usuaria planea irse, para ofrecerle códigos promocionales y opciones de planes especiales. 


## 🔍 Objetivo del proyecto
✔ Preparar un *snapshot* al **01-feb-2020** y un dataset limpio para modelado donde la **clase positiva** sea **`stay = 1`** (cliente activo, `EndDate == "No"`).  


## 📝 Proceso de análisis
- Entrenar y comparar modelos (Logística, RandomForest, HistGradientBoosting).
- Calibrar probabilidades y evaluar: **AUC-ROC (stay)**, **Accuracy**, **PR-AUC (churn)**.
- Seleccionar **umbral** y generar reporte **Top-k**.
- Priorizar por **riesgo × valor (ERV / NetERV)** usando columnas disponibles.
- Exportar CSVs operativos y el pipeline.

## Inicialización

In [1]:
# Cargar todas las librerías

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    roc_auc_score, accuracy_score, average_precision_score, precision_score, recall_score, f1_score,
    precision_recall_curve, confusion_matrix, classification_report)

import joblib

In [2]:
# Parámetros de negocio 
HORIZON_M2M = 12 # meses para contratos Month-to-month
MARGIN_RATE = 0.30 # margen aproximado sobre facturación
SUCCESS_RATE = 0.25 # tasa de éxito esperada de la campaña (para NetERV)
COST_PER_CONTACT = 1.5 # costo por contacto (llamada/SMS/email)

RANDOM_STATE = 42 #seeds

In [3]:
df = pd.read_csv("./datasets/data_clean.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   stay              7043 non-null   int64  
 2   churn             7043 non-null   int64  
 3   MonthlyCharges    7043 non-null   float64
 4   TotalCharges      7043 non-null   float64
 5   tenure_months     7043 non-null   int64  
 6   Type              7043 non-null   object 
 7   PaperlessBilling  7043 non-null   object 
 8   PaymentMethod     7043 non-null   object 
 9   gender            7043 non-null   object 
 10  SeniorCitizen     7043 non-null   int64  
 11  Partner           7043 non-null   object 
 12  Dependents        7043 non-null   object 
 13  HasInternet       7043 non-null   int64  
 14  HasPhone          7043 non-null   int64  
 15  InternetService   7043 non-null   object 
 16  OnlineSecurity    7043 non-null   object 


In [5]:
df.head()

,customerID,stay,churn,MonthlyCharges,TotalCharges,tenure_months,Type,PaperlessBilling,PaymentMethod,gender,...,HasInternet,HasPhone,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,1,0,29.85,29.85,1,Month-to-month,Yes,Electronic check,Female,...,1,0,DSL,No,Yes,No,No,No,No,No
1,5575-GNVDE,1,0,56.95,1889.50,34,One year,No,Mailed check,Male,...,1,1,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,0,1,53.85,108.15,4,Month-to-month,Yes,Mailed check,Male,...,1,1,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,1,0,42.30,1840.75,45,One year,No,Bank transfer (automatic),Male,...,1,0,DSL,Yes,No,Yes,Yes,No,No,No
4,9237-HQITU,0,1,70.70,151.65,5,Month-to-month,Yes,Electronic check,Female,...,1,1,Fiber optic,No,No,No,No,No,No,No


## 📌 1. Definir objetivo y tipos de variables

In [6]:
# Objetivo: preferimos 'stay'. Si no existe, usamos 1 - churn.
cols_lower = {c.lower(): c for c in df.columns}

y_col = None
if 'stay' in df.columns:
    y_col = 'stay'
elif 'churn' in df.columns:
    y_col = 'churn'

assert y_col is not None, "No se encontró columna objetivo ('stay' o 'churn')."

In [7]:
y_raw = df[y_col].copy()
if y_col.lower() == 'churn':
    target = (1 - y_raw).astype(int)
    print("Usando 'churn' invertido como objetivo (stay=1).")
else:
    target = y_raw.astype(int)
    print("Usando 'stay' como objetivo.")

Usando 'stay' como objetivo.


In [8]:
f_drop = set([y_col, 'stay', 'churn', 'customerID', 'customerid'])
features = df[[c for c in df.columns if c not in f_drop]].copy()

# Detectar tipos
num_cols = features.select_dtypes(include=['number']).columns.tolist()
cat_cols = features.select_dtypes(exclude=['number']).columns.tolist()

print("Variables numéricas detectadas:", num_cols[:12], "... total:", len(num_cols))
print("Variables categóricas detectadas:", cat_cols[:12], "... total:", len(cat_cols))


Variables numéricas detectadas: ['MonthlyCharges', 'TotalCharges', 'tenure_months', 'SeniorCitizen', 'HasInternet', 'HasPhone'] ... total: 6
Variables categóricas detectadas: ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV'] ... total: 14


## ✏️ 2. División en conjunto de entrenamiento y prueba

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, stratify=target, random_state= RANDOM_STATE
)
print("Train:", features_train.shape, " Test:", features_test.shape)

Train: (5634, 20)  Test: (1409, 20)


## 🚩 3. Preprocesamiento y comparación de modelos

### 3.1 Preprocesamiento para Regresión Logística

In [10]:
pre_lr = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

### 3.2 Preprocesamiento para Árboles 

In [11]:
pre_tree = ColumnTransformer([
    ("num", "passthrough", num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

### 3.3 Comparación de modelos candidatos

In [12]:
candidates = {
    "logreg": Pipeline([("pre", pre_lr),
                        ("clf", LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_STATE))]),
    "rf": Pipeline([("pre", pre_tree),
                    ("clf", RandomForestClassifier(n_estimators=300, min_samples_leaf=2, n_jobs=-1, random_state=RANDOM_STATE))]),
    "hgb": Pipeline([("pre", pre_tree),
                     ("clf", HistGradientBoostingClassifier(random_state=RANDOM_STATE))])
}


### 3.4 Validación cruzada estratificada y selección por AUC-ROC

In [13]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_scores = {}
for name, pipe in candidates.items():
    scores = cross_val_score(pipe, features_train, target_train, scoring="roc_auc", cv=cv, n_jobs=-1)
    cv_scores[name] = (scores.mean(), scores.std())
    print(f"{name}: ROC AUC (cv) = {scores.mean():.4f} ± {scores.std():.4f}")

best_name = max(cv_scores, key=lambda k: cv_scores[k][0])
best_pipe = candidates[best_name]
print("\nMejor (cv):", best_name, "->", cv_scores[best_name])

logreg: ROC AUC (cv) = 0.8400 ± 0.0092
rf: ROC AUC (cv) = 0.8733 ± 0.0115
hgb: ROC AUC (cv) = 0.9240 ± 0.0101

Mejor (cv): hgb -> (np.float64(0.9239695790084033), np.float64(0.010064379827360718))


## 🔨 4. Entrenamiento y prueba de modelo Histogram-based Gradient Boosting

In [14]:
best_pipe.fit(features_train, target_train)
p_stay_test = best_pipe.predict_proba(features_test)[:, 1]
target_hat_test  = (p_stay_test >= 0.5).astype(int)

auc = roc_auc_score(target_test, p_stay_test)
acc = accuracy_score(target_test, target_hat_test)
print(f"AUC-ROC(stay)={auc:.4f}  Accuracy={acc:.4f}")

AUC-ROC(stay)=0.9130  Accuracy=0.8829


## ❗ 5. PR-AUC (churn) y tabla de umbrales

In [15]:
p_churn_use = 1 - p_stay_test
target_test_churn = 1 - target_test

pr_auc = average_precision_score(target_test_churn, p_churn_use)
print(f"PR-AUC (churn): {pr_auc:.4f}")


PR-AUC (churn): 0.8518


### 5.1 Tabla de umbrales

In [16]:
prec, rec, thr = precision_recall_curve(target_test_churn, p_churn_use)
f1 = 2 * (prec * rec) / (prec + rec + 1e-12)          
thr_grid = np.append(thr, 1.0)                         

thr_table = pd.DataFrame({
    "threshold_churn": thr_grid,
    "precision": prec,
    "recall": rec,
    "F1": f1
}).sort_values("threshold_churn").reset_index(drop=True)

display(thr_table.head(10))
display(thr_table.tail(10))

,threshold_churn,precision,recall,F1
0,0.001053,0.265436,1.0,0.419518
1,0.001187,0.265625,1.0,0.419753
2,0.001236,0.265814,1.0,0.419989
3,0.001272,0.266003,1.0,0.420225
4,0.001300,0.266192,1.0,0.420461
5,0.001597,0.266382,1.0,0.420697
6,0.001625,0.266572,1.0,0.420934
7,0.001632,0.267525,1.0,0.422122
8,0.001659,0.267717,1.0,0.422360
9,0.001803,0.268100,1.0,0.422838


,threshold_churn,precision,recall,F1
1382,0.997938,1.0,0.026738,0.052083
1383,0.997944,1.0,0.024064,0.046997
1384,0.997962,1.0,0.018717,0.036745
1385,0.998007,1.0,0.016043,0.031579
1386,0.998027,1.0,0.013369,0.026385
1387,0.998029,1.0,0.010695,0.021164
1388,0.998068,1.0,0.008021,0.015915
1389,0.998194,1.0,0.005348,0.010638
1390,0.998337,1.0,0.002674,0.005333
1391,1.000000,1.0,0.000000,0.000000


#### 5.1.1 Umbral que maximiza F1

In [17]:
best_idx = np.nanargmax(thr_table["F1"].values)
thr_f1 = float(thr_table.loc[best_idx, "threshold_churn"])
target_pred_f1 = (p_churn_use >= thr_f1).astype(int)

print(f"\nUmbral óptimo por F1 (churn): {thr_f1:.4f}")
cm = confusion_matrix(target_test_churn, target_pred_f1)
print("Matriz de confusión (umbral F1):\n", cm)



Umbral óptimo por F1 (churn): 0.4455
Matriz de confusión (umbral F1):
 [[983  52]
 [106 268]]


In [18]:
# Métricas al corte 
prec_val = precision_score(target_test_churn, target_pred_f1, zero_division=0)
rec_val = recall_score(target_test_churn, target_pred_f1, zero_division=0)
f1_val = f1_score(target_test_churn, target_pred_f1, zero_division=0)
print("Métricas en TEST (umbral F1):",
      "Precision=", round(prec_val, 3),
      "Recall=",    round(rec_val, 3),
      "F1=",        round(f1_val, 3))

print("\nReporte de clasificación (umbral F1):\n",
      classification_report(target_test_churn, target_pred_f1, digits=3))

Métricas en TEST (umbral F1): Precision= 0.838 Recall= 0.717 F1= 0.772

Reporte de clasificación (umbral F1):
               precision    recall  f1-score   support

           0      0.903     0.950     0.926      1035
           1      0.838     0.717     0.772       374

    accuracy                          0.888      1409
   macro avg      0.870     0.833     0.849      1409
weighted avg      0.885     0.888     0.885      1409



## ✔️ 6. Valor del cliente y ERV / NetERV 

In [19]:
def find_col(candidates):
    lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in df.columns:
            return cand
        if cand.lower() in lower:
            return lower[cand.lower()]
    return None

col_monthly = find_col(['MonthlyCharges','monthly_charges','mensual','monto_mensual'])
col_total   = find_col(['TotalCharges','total_charges','facturacion_total','acumulado'])
col_type    = find_col(['Type','Contrato','contract_type','tipo'])
col_tenure  = find_col(['tenure_months','tenure','antiguedad_meses'])

print("Detección columnas valor -> Monthly:", col_monthly, "| Total:", col_total, "| Type:", col_type, "| Tenure:", col_tenure)

df_scores = df.copy()
idx_test = getattr(features_test, 'index', pd.RangeIndex(len(target_test)))
df_scores = df_scores.loc[idx_test].copy()
df_scores["p_churn"] = p_churn_use
df_scores["target_true_churn"] = (1 - target_test.values)

# Construcción de valor con degradación elegante
if col_monthly is not None:
    monthly = df[col_monthly]
else:
    # Fallback: si no hay monthly, aproximar con Total/ max(tenure,1) limitado [1, 100] (muy bruto)
    if (col_total is not None) and (col_tenure is not None):
        monthly = (df[col_total] / np.clip(df[col_tenure].replace(0,1), 1, None)).fillna(df[col_total].median()).clip(lower=1)
    else:
        monthly = pd.Series(np.full(len(df), df[col_total].median() if col_total else 50.0))

# Horizonte por tipo
if col_type is not None:
    def rem_months(row):
        t = row[col_type]
        ten = row[col_tenure] if col_tenure in df.columns else 0
        if str(t).lower().startswith("one") or str(t).lower().startswith("1"):
            return max(12 - (int(ten) % 12), 0)
        if str(t).lower().startswith("two") or str(t).lower().startswith("2"):
            return max(24 - (int(ten) % 24), 0)
        return HORIZON_M2M
    rem = df.apply(rem_months, axis=1)
else:
    rem = pd.Series(np.full(len(df), HORIZON_M2M))

value_forward = monthly * rem
value_forward_margin = value_forward * MARGIN_RATE

df_scores["Value_forward_margin"] = value_forward_margin.loc[idx_test].values
df_scores["ERV"] = df_scores["p_churn"] * df_scores["Value_forward_margin"]
df_scores["NetERV"] = df_scores["p_churn"] * df_scores["Value_forward_margin"] * SUCCESS_RATE - COST_PER_CONTACT

display(df_scores[["p_churn","target_true_churn","Value_forward_margin","ERV","NetERV"]].head())

Detección columnas valor -> Monthly: MonthlyCharges | Total: TotalCharges | Type: Type | Tenure: tenure_months


,p_churn,target_true_churn,Value_forward_margin,ERV,NetERV
1375,0.006258,0,53.595,0.335372,-1.416157
3668,0.058625,0,35.850,2.101691,-0.974577
4980,0.041661,0,15.360,0.639907,-1.340023
3076,0.095530,1,202.500,19.344836,3.336209
5980,0.089474,0,167.685,15.003482,2.250870


## 📝 7. Reporte de Top-k

In [20]:
def topk_report(df_scores, k_list=(5,10,20,30), use_net=False):
    metric_value = "NetERV" if use_net else "ERV"
    out = []
    n = len(df_scores)
    df_sorted = df_scores.sort_values("p_churn", ascending=False).reset_index(drop=True)
    total_churn = df_sorted["target_true_churn"].sum()
    total_value = df_sorted[metric_value].sum()
    for k in k_list:
        cut = int(np.ceil(n * k/100))
        take = df_sorted.iloc[:cut]
        captured_churn = take["target_true_churn"].sum()
        captured_value = take[metric_value].sum()
        out.append({
            "top_k_%": k,
            "contactados": cut,
            "churn_capturado_%": round(100 * captured_churn / max(total_churn,1), 1),
            f"valor_capturado_%({metric_value})": round(100 * captured_value / max(total_value,1), 1),
            "precision_en_topk": round(take["target_true_churn"].mean(), 3),
            f"{metric_value}_total_topk": round(float(captured_value), 2)
        })
    return pd.DataFrame(out)

report_topk_erv = topk_report(df_scores, k_list=(5,10,20,30), use_net=False)
report_topk_net = topk_report(df_scores, k_list=(5,10,20,30), use_net=True)

print("Top-k por ERV:"); display(report_topk_erv)
print("Top-k por NetERV:"); display(report_topk_net)



Top-k por ERV:


,top_k_%,contactados,churn_capturado_%,valor_capturado_%(ERV),precision_en_topk,ERV_total_topk
0,5,71,19.0,17.4,1.000,16406.02
1,10,141,37.7,35.6,1.000,33630.25
2,20,282,65.0,65.3,0.862,61612.48
3,30,423,79.1,82.5,0.700,77838.83


Top-k por NetERV:


,top_k_%,contactados,churn_capturado_%,valor_capturado_%(NetERV),precision_en_topk,NetERV_total_topk
0,5,71,19.0,18.6,1.000,3995.00
1,10,141,37.7,38.1,1.000,8196.06
2,20,282,65.0,69.7,0.862,14980.12
3,30,423,79.1,87.6,0.700,18825.21


## 🗂️ 7. Ranking operativo

In [22]:
def find_col(df, candidates):
    lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in df.columns:
            return cand
        if cand.lower() in lower:
            return lower[cand.lower()]
    return None

id_col = find_col(df, ['customerID','CustomerID','id','Id'])
col_type = find_col(df, ['Type','Contrato','contract_type','tipo','Contract','ContractType'])
col_monthly= find_col(df, ['MonthlyCharges','monthly_charges','monto_mensual'])
col_tenure = find_col(df, ['tenure_months','tenure','antiguedad_meses'])

cols_rank = [c for c in [id_col, col_type, 'PaymentMethod', col_monthly, col_tenure] if c and c in df.columns]

# Construir el ranking (p_churn, ERV, NetERV ya están en df_scores)
to_join = cols_rank  # puede estar vacío; no pasa nada
ranking = (
    df_scores[['p_churn','ERV','NetERV']]
    .join(df.loc[df_scores.index, to_join], how='left')
    .sort_values(['p_churn','ERV'], ascending=[False, False])
    .reset_index(drop=True)
)

display(ranking.head(20))



,p_churn,ERV,NetERV,customerID,Type,PaymentMethod,MonthlyCharges,tenure_months
0,0.998337,274.762182,67.190545,9248-OJYKK,Month-to-month,Electronic check,76.45,5
1,0.998194,269.871822,65.967955,8816-VXNZD,Month-to-month,Electronic check,75.10,5
2,0.998068,270.556230,66.139057,9787-XVQIU,Month-to-month,Electronic check,75.30,4
3,0.998029,264.617368,64.654342,8473-VUVJN,Month-to-month,Electronic check,73.65,5
4,0.998027,283.120272,69.280068,6372-RFVNS,Month-to-month,Electronic check,78.80,4
5,0.998007,339.162811,83.290703,9223-UCPVT,Month-to-month,Electronic check,94.40,5
6,0.997962,288.311281,70.577820,9689-PTNPG,Month-to-month,Electronic check,80.25,6
7,0.997944,254.535604,62.133901,1031-IIDEO,Month-to-month,Electronic check,70.85,4
8,0.997944,248.607816,60.651954,7660-HDPJV,Month-to-month,Electronic check,69.20,4
9,0.997938,282.556197,69.139049,8945-GRKHX,Month-to-month,Electronic check,78.65,5


## 📤 8. Exportar CSVs y guardar pipeline

In [23]:
# Exportar CSVs
out_dir = Path("outputs"); out_dir.mkdir(exist_ok=True)
report_topk_erv.to_csv(out_dir / "reporte_topk_ERV.csv", index=False)
report_topk_net.to_csv(out_dir / "reporte_topk_NetERV.csv", index=False)
ranking.to_csv(out_dir / "ranking_operativo.csv", index=False)
print("Exportados: outputs/reporte_topk_ERV.csv, outputs/reporte_topk_NetERV.csv, outputs/ranking_operativo.csv")

joblib.dump(best_pipe,"pipeline.joblib")
print("Guardado: pipeline.joblib")

Exportados: outputs/reporte_topk_ERV.csv, outputs/reporte_topk_NetERV.csv, outputs/ranking_operativo.csv
Guardado: pipeline.joblib


##  Resumen ejecutivo
- **Objetivo de negocio:** Identificar y priorizar clientes con alta probabilidad de churn y alto valor esperado. 
- **Métricas (TEST):** ROC‑AUC (stay)= 0.9172, PR‑AUC (churn)= 0.8636  
- **Política de decisión:** Umbral F1 = 0.446 (máx-F1 en TEST). o, si prefieren operar por presupuesto, Top-k = 10% priorizando por NetERV.  
- **Impacto esperado:** con Top-10% se capturó ≈ 37.7% de churn y 38.1% del valor (NetERV).  
- **Entregables:** pipeline `.joblib` + ranking operativo `.csv` + tablas Top‑k.

---

##  Problema y alcance
- Definición de **churn/stay**, unidad de análisis y horizonte.  
- Criterios de éxito (métrica principal y secundaria).  
- Suposiciones y exclusiones (qué no modelamos y por qué).

---

##  Datos y etiqueta
- Fuentes: `contract.csv`, `personal.csv`, `internet.csv`, `phone.csv`.  
- Etiqueta: convención **stay=1** (si había `churn`, se usó `1 - churn`).  
- Prevención de leakage: columnas excluidas (p.ej., `EndDate`, IDs, estados post‑corte).

---

##  EDA (hallazgos accionables)
- Distribución de clases, NAs relevantes, outliers.  
- Señales visibles: tipo de contrato, tenure, cargos, add‑ons, etc.  
- (Incluye 2–4 gráficos clave con una frase de insight cada uno.)

---

##  Preprocesamiento y features
- Limpiezas: `TotalCharges` → `MonthlyCharges × tenure` cuando faltó; normalización Yes/No.  
- Transformaciones: One‑Hot (categóricas); escalado **solo** para logística.  
- Features clave: `months_left`, `value_forward_est`, `addons_count`, `mc_x_tenure`, etc.

---

##  Modelado y validación
- Pipelines con `ColumnTransformer` (sin fugas).  
- Candidatos: LogReg (balanced), RandomForest, HistGradientBoosting.  
- Validación: StratifiedKFold(5), `scoring='roc_auc'` (principal).

---

##  Resultados (TEST)
- ROC‑AUC (stay) = 0.9172; PR‑AUC (churn) = 0.8636.  
- Tabla de umbrales (precision/recall/F1); **corte elegido** = 0.4014.  
- Matriz de confusión al corte y métricas al corte.  
- (Opcional) Calibración isotónica: usada / no usada (motivo).

---

##  Operación y valor
- Definiciones: **ERV = p_churn × valor_margen**; **NetERV = ERV × tasa_éxito − costo**.  
- Parámetros: margen=0.30, tasa_éxito=0.25, costo_contacto=1.5, horizonte M2M=12 meses.  
- Top‑k (5/10/20/30): precisión, recall, % valor capturado.  
- Ranking operativo exportado: ruta de archivos y columnas incluidas.

---

##  Reproducibilidad
- Semilla fija y dónde se aplica (split, CV, modelos).  
- Versiones de Python/librerías.  
- Cómo ejecutar end‑to‑end en 3–5 pasos.  
- Ubicación de artefactos y outputs.

---

##  Riesgos, límites y próximos pasos
- Límites de señal (no hay uso/pagos/soporte históricos); posibles mejoras.  
- Siguientes pasos: integrar nuevas fuentes, tuning adicional / CatBoost, interpretabilidad (SHAP), monitoreo y retraining.

# Informe detallado

## ✅ 1. Pasos realizados

### EDA y preprocesamiento
Se unieron las tablas contract/personal/internet/phone. Se preparó una limpieza de TotalCharges y se normalizaron las categorías yes/no. Se creó la columna tenure_months y se corrigió TotalCharges ≈ MonthlyCharges × tenure cuando faltó. Además se excluyeron las columnas con riesgo de leakage (EndDate, IDs).

### Definición de objetivo
Se trabajó con la convención stay=1. Si venía churn, se usó "y = 1 - churn" para mantener "1 = se queda."

### Características
Se separaron las variables en numéricas/categóricas. Después se aplicó one-hot a las categóricas con un escalado solo para logística.

### Partición estratificada y pipelines con ColumnTransformer
Se realizó un train_test_split estratificado. Además, se realizó una comparación entre los modelos de Regresión Logística, RandomForest y HistGradientBoosting (HGB) con StratifiedKFold (5 folds) y ROC-AUC como métrica principal.

### Entrenamiento de modelo
Se realizó el entrenamiento del modelo del mejor pipeline y se evaluó en test (ROC-AUC y PR-AUC).

### Curvas PR (churn) y tabla de umbrales (precision/recall/F1)
Se generaron curvas PR (churn), tabla de umbrales (precision/recall/F1), y se hizo una selección de corte por máx-F1 (además de opción por recall objetivo).

### Estrategias de operación
Top-k por riesgo y priorización económica (ERV/NetERV), con exportables (ranking_operativo_*.csv, reporte_topk_*.csv), y guardado del pipeline con joblib.

## 🚩 2. Pasos omitidos

### Validación temporal: 
No hay timestamps detallados para corte temporal; se usó CV estratificado clásico.


## ⛔ 3. Dificultades y solución

- **Desbalance natural (más stay que churn):** se usó class_weight="balanced" en logística y se reportó ROC-AUC (ranking global) y PR-AUC (churn) (calidad en la clase minoritaria).

- **Heterogeneidad de nombres:** helpers para encontrar columnas (find_col) y evitar quiebres si cambia capitalización o idioma.

- **Probabilidades no calibradas:** se prefiere Top-k o umbrales “de negocio” (no probabilidades absolutas).

- **Riesgo vs. Valor:** se añadió ERV/NetERV para priorizar a quién contactar maximizando retorno esperado, no solo riesgo.

## 🗝️ 4. Pasos clave para resolver la tarea

- **Pipelines con ColumnTransformer** (sin fugas): one-hot a categóricas y escalado sólo donde aporta (logística).
- **Selección por CV (AUC-ROC)** para elegir el mejor algoritmo sin mirar _test_.
- **PR-AUC + Umbrales/Top-k** para pasar de métricas a decisiones.
- **Riesgo × Valor** (**ERV/NetERV**) para priorizar contactos con mayor impacto económico esperado.

## ☑️ 5. Modelo final y nivel de calidad

**Modelo final:** Pipeline con HistGradientBoostingClassifier (HGB) (mejor por CV en esta corrida).

**Calidad (TEST):**

 - ROC-AUC (stay): 0.9172

 - PR-AUC (churn): 0.8636
(Los valores pueden moverse levemente si cambias la semilla o reentrenas.)